In [ ]:
# Electricity Demand Forecasting
# Next-day Hourly Electricity Demand Prediction using Weather and Temporal Features

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# For data preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV

# For modeling
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX

# For neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# For ensemble stacking
from sklearn.ensemble import StackingRegressor

# For visualization
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set plot styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")

# 1. Data Loading and Initial Exploration
print("Loading data...")
df = pd.read_csv('merged_weather_demand_final.csv')

# Convert DateTime to datetime format
df['DateTime'] = pd.to_datetime(df['DateTime'])
print(f"Dataset period: {df['DateTime'].min()} to {df['DateTime'].max()}")
print(f"Dataset shape: {df.shape}")


# Display basic info
print("\nDataset Info:")
print(df.info())

print("\nStatistical Summary:")
print(df.describe())

print("\nChecking for missing values:")
print(df.isnull().sum())

# Handle missing values if any
if df.isnull().sum().sum() > 0:
    print("\nHandling missing values...")
    # For numeric columns, fill with median
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    # For categorical columns, fill with mode
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mode()[0])

    print("After handling missing values:")
    print(df.isnull().sum().sum())



# 3. Feature Engineering

print("\nPerforming feature engineering...")

# 3.1 Create lag features (previous days demand)
df_features = df.copy().sort_values('DateTime')

# Add lag features (previous 24 hours demand) 
df_features['demand_lag_24h'] = df_features['Demand (MW)'].shift(24)

# Add lag features for previous day same hour (24, 48, 72 hours)
for i in range(1, 4):
    df_features[f'demand_lag_{i}d_same_hour'] = df_features['Demand (MW)'].shift(i*24)

# Add moving averages for smoothed signals
df_features['demand_ma_24h'] = df_features['Demand (MW)'].rolling(window=24).mean()
df_features['demand_ma_7d'] = df_features['Demand (MW)'].rolling(window=24*7).mean()

# Add daily min, max, and range
df_features['day_of_year'] = df_features['DateTime'].dt.dayofyear
daily_stats = df_features.groupby('day_of_year')['Demand (MW)'].agg(['min', 'max'])
daily_stats['range'] = daily_stats['max'] - daily_stats['min']
df_features = df_features.merge(daily_stats, on='day_of_year', how='left')
df_features.rename(columns={'min': 'daily_min', 'max': 'daily_max', 'range': 'daily_range'}, inplace=True)

# Add Cyclical Features for hour, day of week, month, season
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['Hour']/24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['Hour']/24)
df_features['weekday_sin'] = np.sin(2 * np.pi * df_features['Weekday']/7)
df_features['weekday_cos'] = np.cos(2 * np.pi * df_features['Weekday']/7)
df_features['month_sin'] = np.sin(2 * np.pi * df_features['Month']/12)
df_features['month_cos'] = np.cos(2 * np.pi * df_features['Month']/12)

# Weather derivatives
df_features['temp_change_24h'] = df_features['Temperature (F)'] - df_features['Temperature (F)'].shift(24)

# Create peak/off-peak indicator (6am-10pm is peak)
df_features['is_peak_hours'] = ((df_features['Hour'] >= 6) & (df_features['Hour'] <= 22)).astype(int)

# Drop rows with NaN due to lag features
df_features.dropna(inplace=True)
print(f"After feature engineering: {df_features.shape}")

# 4. Data Preparation for Modeling

# 4.1 Define target variable and features
target = 'Demand (MW)'

# 4.2 Select relevant features for modeling
categorical_features = ['Weekday', 'Month', 'Season', 'Sub-Region']
numerical_features = [
    'Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure', 'UV Index',
    'Cloud Cover', 'Hour', 'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos',
    'month_sin', 'month_cos', 'is_peak_hours', 'demand_lag_24h',
    'demand_lag_1d_same_hour', 'demand_lag_2d_same_hour', 'demand_lag_3d_same_hour',
    'temp_change_24h', 'demand_ma_24h', 'demand_ma_7d'
]

all_features = numerical_features + categorical_features

# 4.3 Train-test split (by date to avoid data leakage)
split_date = df_features['DateTime'].max() - timedelta(days=60)  # Last 60 days as test set
train_df = df_features[df_features['DateTime'] <= split_date]
test_df = df_features[df_features['DateTime'] > split_date]

print(f"Training data: {train_df.shape} (From {train_df['DateTime'].min()} to {train_df['DateTime'].max()})")
print(f"Testing data: {test_df.shape} (From {test_df['DateTime'].min()} to {test_df['DateTime'].max()})")

# 4.4 Define preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4.5 Prepare X and y
X_train = train_df[all_features]
y_train = train_df[target]
X_test = test_df[all_features]
y_test = test_df[target]

# Store DateTime for plotting
test_dates = test_df['DateTime']

# 5. Model Implementation and Evaluation

# Evaluation function
def evaluate_model(model, X_test, y_test, model_name, naive_mae=None):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)

    print(f"\n{model_name} Performance:")
    print(f"MAE: {mae:.2f} MW")
    print(f"RMSE: {rmse:.2f} MW")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²: {r2:.4f}")

    if naive_mae:
        improvement = ((naive_mae - mae) / naive_mae) * 100
        print(f"Improvement over naive baseline: {improvement:.2f}%")

    # Plot actual vs predicted
    plt.figure(figsize=(14, 7))
    plt.plot(test_dates, y_test, label='Actual Demand', alpha=0.7)
    plt.plot(test_dates, y_pred, label='Predicted Demand', alpha=0.7)
    plt.title(f'{model_name}: Actual vs Predicted Demand')
    plt.xlabel('Date')
    plt.ylabel('Demand (MW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{model_name.lower().replace(" ", "_")}_prediction.png')

    # Return performance metrics and predictions
    return {
        'model_name': model_name,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'r2': r2,
        'y_pred': y_pred
    }

print("\nImplementing forecasting models...")

# 5.1 Naive Baseline (Previous day's same hour)
print("\nCreating naive baseline model...")

# Create a test set with the naive prediction (previous day same hour)
naive_pred = test_df['demand_lag_1d_same_hour'].values
naive_mae = mean_absolute_error(y_test, naive_pred)
naive_rmse = np.sqrt(mean_squared_error(y_test, naive_pred))
naive_mape = np.mean(np.abs((y_test - naive_pred) / y_test)) * 100

print(f"\nNaive Baseline Performance:")
print(f"MAE: {naive_mae:.2f} MW")
print(f"RMSE: {naive_rmse:.2f} MW")
print(f"MAPE: {naive_mape:.2f}%")

# Plot baseline
plt.figure(figsize=(14, 7))
plt.plot(test_dates, y_test, label='Actual Demand', alpha=0.7)
plt.plot(test_dates, naive_pred, label='Naive Forecast', alpha=0.7)
plt.title('Naive Baseline: Actual vs Predicted Demand')
plt.xlabel('Date')
plt.ylabel('Demand (MW)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('naive_baseline_prediction.png')

# Store results for comparison
results = []
predictions = {}
predictions['Naive Baseline'] = naive_pred

# 5.2 Linear Regression
print("\nTraining Linear Regression model...")
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Ridge(alpha=0.1))  # Ridge regression for better stability
])
lr_pipeline.fit(X_train, y_train)
lr_results = evaluate_model(lr_pipeline, X_test, y_test, "Linear Regression", naive_mae)
results.append(lr_results)
predictions['Linear Regression'] = lr_results['y_pred']

# 5.3 Random Forest Regressor
print("\nTraining Random Forest model...")
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1))
])
rf_pipeline.fit(X_train, y_train)
rf_results = evaluate_model(rf_pipeline, X_test, y_test, "Random Forest", naive_mae)
results.append(rf_results)
predictions['Random Forest'] = rf_results['y_pred']

# 5.4 Gradient Boosting
print("\nTraining Gradient Boosting model...")
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42))
])
gb_pipeline.fit(X_train, y_train)
gb_results = evaluate_model(gb_pipeline, X_test, y_test, "Gradient Boosting", naive_mae)
results.append(gb_results)
predictions['Gradient Boosting'] = gb_results['y_pred']

# 5.5 XGBoost
print("\nTraining XGBoost model...")
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1,
                              random_state=42, n_jobs=-1))
])
xgb_pipeline.fit(X_train, y_train)
xgb_results = evaluate_model(xgb_pipeline, X_test, y_test, "XGBoost", naive_mae)
results.append(xgb_results)
predictions['XGBoost'] = xgb_results['y_pred']

# 5.6 Simple Neural Network (Feedforward)

try:
    print("\nTraining Feedforward Neural Network...")

    # Preprocess data
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)

    # Build model
    nn_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    # Compile and train
    nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    nn_model.fit(
        X_train_preprocessed, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )

    # Make predictions
    nn_pred = nn_model.predict(X_test_preprocessed).flatten()

    # Calculate metrics
    nn_mae = mean_absolute_error(y_test, nn_pred)
    nn_rmse = np.sqrt(mean_squared_error(y_test, nn_pred))
    nn_mape = np.mean(np.abs((y_test - nn_pred) / y_test)) * 100
    nn_r2 = r2_score(y_test, nn_pred)

    print(f"\nNeural Network Performance:")
    print(f"MAE: {nn_mae:.2f} MW")
    print(f"RMSE: {nn_rmse:.2f} MW")
    print(f"MAPE: {nn_mape:.2f}%")
    print(f"R²: {nn_r2:.4f}")

    # Calculate improvement over baseline
    nn_improvement = ((naive_mae - nn_mae) / naive_mae) * 100
    print(f"Improvement over naive baseline: {nn_improvement:.2f}%")

    # Plot actual vs predicted
    plt.figure(figsize=(14, 7))
    plt.plot(test_dates, y_test, label='Actual Demand', alpha=0.7)
    plt.plot(test_dates, nn_pred, label='Predicted Demand', alpha=0.7)
    plt.title('Neural Network: Actual vs Predicted Demand')
    plt.xlabel('Date')
    plt.ylabel('Demand (MW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('neural_network_prediction.png')

    # Store results
    nn_results = {
        'model_name': 'Neural Network',
        'mae': nn_mae,
        'rmse': nn_rmse,
        'mape': nn_mape,
        'r2': nn_r2,
        'y_pred': nn_pred
    }
    results.append(nn_results)
    predictions['Neural Network'] = nn_pred

except Exception as e:
    print(f"Error training neural network: {e}")
    pass

# 6. Ensemble Models (Requirement)

print("\nImplementing ensemble models...")

# 6.1 Stacking Ensemble
print("\nTraining Stacking Ensemble model...")

# Define base models for stacking
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)),
    ('xgb', xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42))
]

# Define meta-model
meta_model = Ridge(alpha=0.1)

# Create stacking ensemble
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

# Create pipeline with preprocessor
stacking_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', stacking_model)
])

# Train stacking ensemble
stacking_pipeline.fit(X_train, y_train)

# Evaluate stacking ensemble
stacking_results = evaluate_model(stacking_pipeline, X_test, y_test, "Stacking Ensemble", naive_mae)
results.append(stacking_results)
predictions['Stacking Ensemble'] = stacking_results['y_pred']

# 7. Model Comparison

print("\nComparing all models...")

# Create a dataframe with all results
results_df = pd.DataFrame(results)
print("\nModel Performance Summary:")
print(results_df[['model_name', 'mae', 'rmse', 'mape', 'r2']])

# Find best model
best_model_idx = results_df['mae'].idxmin()
best_model = results_df.loc[best_model_idx, 'model_name']
print(f"\nBest model based on MAE: {best_model}")

# Calculate improvements over naive baseline
results_df['improvement_over_naive'] = ((naive_mae - results_df['mae']) / naive_mae) * 100

# Plot MAE comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='model_name', y='mae', data=results_df)
plt.title('MAE Comparison Across Models')
plt.xlabel('Model')
plt.ylabel('Mean Absolute Error (MW)')
plt.axhline(y=naive_mae, color='r', linestyle='--', label=f'Naive Baseline: {naive_mae:.2f}')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_mae_comparison.png')

# Plot MAPE comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='model_name', y='mape', data=results_df)
plt.title('MAPE Comparison Across Models')
plt.xlabel('Model')
plt.ylabel('Mean Absolute Percentage Error (%)')
plt.axhline(y=naive_mape, color='r', linestyle='--', label=f'Naive Baseline: {naive_mape:.2f}%')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_mape_comparison.png')

# Plot R² comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='model_name', y='r2', data=results_df)
plt.title('R² Comparison Across Models')
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_r2_comparison.png')

# Plot improvement over naive baseline
plt.figure(figsize=(12, 6))
sns.barplot(x='model_name', y='improvement_over_naive', data=results_df)
plt.title('Improvement Over Naive Baseline')
plt.xlabel('Model')
plt.ylabel('Improvement (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('model_improvement_comparison.png')

# 8. Feature Importance Analysis
print("\nAnalyzing feature importance...")

# Extract feature names from the preprocessor
feature_names = []
for name, transformer, features in preprocessor.transformers_:
    if name == 'num':
        feature_names.extend(features)
    elif name == 'cat':
        # For categorical features, get the one-hot encoded feature names
        for cat_feature in features:
            categories = train_df[cat_feature].unique()
            for category in categories:
                feature_names.append(f"{cat_feature}_{category}")

# Get feature importance from Random Forest and XGBoost
try:
    # For Random Forest
    rf_model = rf_pipeline.named_steps['model']
    rf_importances = pd.DataFrame({
        'feature': feature_names[:len(rf_model.feature_importances_)],
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False).head(20)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=rf_importances)
    plt.title('Random Forest Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('rf_feature_importance.png')

    # For XGBoost
    xgb_model = xgb_pipeline.named_steps['model']
    xgb_importances = pd.DataFrame({
        'feature': feature_names[:len(xgb_model.feature_importances_)],
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False).head(20)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=xgb_importances)
    plt.title('XGBoost Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('xgb_feature_importance.png')

except Exception as e:
    print(f"Error in feature importance analysis: {e}")

# 9. Visualize predictions from all models in one plot
plt.figure(figsize=(14, 8))
plt.plot(test_dates, y_test, 'k-', label='Actual', linewidth=2, alpha=0.7)

colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown']
color_idx = 0

for model_name, y_pred in predictions.items():
    if model_name == 'Naive Baseline':
        plt.plot(test_dates, y_pred, 'r--', label=model_name, alpha=0.5)
    else:
        if color_idx < len(colors):
            plt.plot(test_dates, y_pred, color=colors[color_idx], label=model_name, alpha=0.6)
            color_idx += 1
        else:
            plt.plot(test_dates, y_pred, label=model_name, alpha=0.6)

plt.title('Demand Predictions Comparison Across Models')
plt.xlabel('Date')
plt.ylabel('Demand (MW)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('all_models_prediction_comparison.png')

# 10. Error Analysis
print("\nPerforming error analysis...")

# Choose the best model for error analysis
best_model_name = best_model
best_model_predictions = predictions[best_model_name]

# Calculate errors
errors = y_test - best_model_predictions
abs_errors = np.abs(errors)
rel_errors = abs_errors / y_test * 100

# Create a DataFrame for error analysis
error_df = pd.DataFrame({
    'DateTime': test_dates,
    'Hour': test_df['Hour'],
    'Weekday': test_df['Weekday'],
    'Month': test_df['Month'],
    'Day': test_df['Day'],
    'Season': test_df['Season'],
    'Temperature (F)': test_df['Temperature (F)'],
    'Actual': y_test,
    'Predicted': best_model_predictions,
    'Error': errors,
    'AbsError': abs_errors,
    'RelError': rel_errors
})

# Error distribution
plt.figure(figsize=(12, 6))
sns.histplot(errors, kde=True, bins=50)
plt.title(f'Distribution of Prediction Errors ({best_model_name})')
plt.xlabel('Error (MW)')
plt.axvline(x=0, color='r', linestyle='--')
plt.grid(True)
plt.tight_layout()
plt.savefig('error_distribution.png')

# Error by hour of day
hourly_error = error_df.groupby('Hour')['AbsError'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.lineplot(x='Hour', y='AbsError', data=hourly_error, marker='o')
plt.title(f'Average Absolute Error by Hour of Day ({best_model_name})')
plt.xlabel('Hour')
plt.ylabel('Mean Absolute Error (MW)')
plt.xticks(range(0, 24))
plt.grid(True)
plt.tight_layout()
plt.savefig('error_by_hour.png')

# Error by day of week
weekday_error = error_df.groupby('Weekday')['AbsError'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='Weekday', y='AbsError', data=weekday_error)
plt.title(f'Average Absolute Error by Day of Week ({best_model_name})')
plt.xlabel('Day of Week (0=Monday, 6=Sunday)')
plt.ylabel('Mean Absolute Error (MW)')
plt.grid(True)
plt.tight_layout()
plt.savefig('error_by_weekday.png')

# Error by temperature
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Temperature (F)', y='AbsError', data=error_df, alpha=0.3)
plt.title(f'Absolute Error vs Temperature ({best_model_name})')
plt.xlabel('Temperature (F)')
plt.ylabel('Absolute Error (MW)')
plt.grid(True)
plt.tight_layout()
plt.savefig('error_vs_temperature.png')

# Relative error by demand level
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Actual', y='RelError', data=error_df, alpha=0.3)
plt.title(f'Relative Error vs Actual Demand ({best_model_name})')
plt.xlabel('Actual Demand (MW)')
plt.ylabel('Relative Error (%)')
plt.grid(True)
plt.tight_layout()
plt.savefig('relative_error_vs_demand.png')

# 11. Next-Day Forecast Simulation (fixed version)
print("\nSimulating next-day hourly forecast...")

# Get the last day in the test set
last_date = test_df['DateTime'].max()
next_day = last_date + timedelta(days=1)
print(f"Simulating forecast for: {next_day.date()}")

# Create a template for next 24 hours
next_day_hours = [next_day.replace(hour=h) for h in range(24)]
next_day_df = pd.DataFrame({'DateTime': next_day_hours})

# Add temporal features
next_day_df['Hour'] = next_day_df['DateTime'].dt.hour
next_day_df['Day'] = next_day_df['DateTime'].dt.day
next_day_df['Month'] = next_day_df['DateTime'].dt.month
next_day_df['Weekday'] = next_day_df['DateTime'].dt.weekday
next_day_df['Weekend'] = next_day_df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

# Determine season (simplistic approach)
def get_season(month):
    if month in [12, 1, 2]:
        return 1  # Winter
    elif month in [3, 4, 5]:
        return 2  # Spring
    elif month in [6, 7, 8]:
        return 3  # Summer
    else:
        return 4  # Fall

next_day_df['Season'] = next_day_df['Month'].apply(get_season)

# For weather data, use the last known values (or you could use weather forecast data if available)

weather_cols = ['Temperature (F)', 'Humidity', 'Wind Speed (mph)', 'Pressure', 'UV Index', 'Cloud Cover']
last_day_weather = df_features[df_features['DateTime'] >= last_date - timedelta(days=1)]

# Interpolate by hour if we have some data for the last day
if len(last_day_weather) > 0:
    for hour in range(24):
        hour_data = last_day_weather[last_day_weather['Hour'] == hour]
        if len(hour_data) > 0:
            for col in weather_cols:
                next_day_df.loc[next_day_df['Hour'] == hour, col] = hour_data[col].values[-1]
        else:
            
            last_week = df_features[(df_features['DateTime'] >= last_date - timedelta(days=7)) &
                                  (df_features['Hour'] == hour)]
            for col in weather_cols:
                next_day_df.loc[next_day_df['Hour'] == hour, col] = last_week[col].mean()

# Add Sub-Region 
next_day_df['Sub-Region'] = test_df['Sub-Region'].iloc[-1]

# Add lag features - only lag 24h
next_day_df['demand_lag_24h'] = test_df['Demand (MW)'].iloc[-24:].values[0] if len(test_df) >= 24 else np.nan

# Add demand lag for previous day same hour (24, 48, 72 hours)
for i in range(1, 4):
    for hour in range(24):
        same_hour_data = test_df[(test_df['DateTime'] <= last_date) & (test_df['Hour'] == hour)]
        if len(same_hour_data) >= i:
            next_day_df.loc[next_day_df['Hour'] == hour, f'demand_lag_{i}d_same_hour'] = same_hour_data['Demand (MW)'].iloc[-i]
        else:
            next_day_df.loc[next_day_df['Hour'] == hour, f'demand_lag_{i}d_same_hour'] = np.nan

# Add moving averages (simplified)
last_24h = test_df['Demand (MW)'].iloc[-24:].mean() if len(test_df) >= 24 else test_df['Demand (MW)'].mean()
last_7d = test_df['Demand (MW)'].iloc[-24*7:].mean() if len(test_df) >= 24*7 else test_df['Demand (MW)'].mean()
next_day_df['demand_ma_24h'] = last_24h
next_day_df['demand_ma_7d'] = last_7d

# Add daily stats (simplified)
last_day_stats = test_df.groupby(test_df['DateTime'].dt.date)['Demand (MW)'].agg(['min', 'max'])
if not last_day_stats.empty:
    next_day_df['daily_min'] = last_day_stats['min'].iloc[-1]
    next_day_df['daily_max'] = last_day_stats['max'].iloc[-1]
    next_day_df['daily_range'] = next_day_df['daily_max'] - next_day_df['daily_min']
else:
    next_day_df['daily_min'] = test_df['Demand (MW)'].min()
    next_day_df['daily_max'] = test_df['Demand (MW)'].max()
    next_day_df['daily_range'] = next_day_df['daily_max'] - next_day_df['daily_min']

# Add Cyclical Features
next_day_df['hour_sin'] = np.sin(2 * np.pi * next_day_df['Hour']/24)
next_day_df['hour_cos'] = np.cos(2 * np.pi * next_day_df['Hour']/24)
next_day_df['weekday_sin'] = np.sin(2 * np.pi * next_day_df['Weekday']/7)
next_day_df['weekday_cos'] = np.cos(2 * np.pi * next_day_df['Weekday']/7)
next_day_df['month_sin'] = np.sin(2 * np.pi * next_day_df['Month']/12)
next_day_df['month_cos'] = np.cos(2 * np.pi * next_day_df['Month']/12)

# Temperature change (simplified)
next_day_df['temp_change_24h'] = 0  # Placeholder

# Create peak/off-peak indicator
next_day_df['is_peak_hours'] = ((next_day_df['Hour'] >= 6) & (next_day_df['Hour'] <= 22)).astype(int)

# Fill missing numerical and categorical values in X_next_day
for col in numerical_features:
    if col in next_day_df.columns:
        next_day_df[col] = next_day_df[col].fillna(X_train[col].median())

for col in categorical_features:
    if col in next_day_df.columns:
        next_day_df[col] = next_day_df[col].fillna(X_train[col].mode()[0])

# Extract features needed for prediction 
X_next_day = next_day_df[all_features]

# Map model names to actual model objects
model_map = {
    "Linear Regression": lr_pipeline,
    "Random Forest": rf_pipeline,
    "Gradient Boosting": gb_pipeline,
    "XGBoost": xgb_pipeline,
    "Stacking Ensemble": stacking_pipeline
}

# Add Neural Network if available
if 'nn_model' in locals() and 'nn_pred' in locals():
    model_map["Neural Network"] = nn_model

# Add SARIMA if available
if 'sarima_results' in locals() and 'sarima_pred' in locals():
    model_map["SARIMA"] = sarima_results

# Select the best model
best_model_name = results_df.loc[results_df['mae'].idxmin(), 'model_name']
best_model = model_map.get(best_model_name)

if best_model is None:
    raise ValueError(f"Best model '{best_model_name}' is not available or not trained.")

# For Neural Network, we need to transform the data differently
if best_model_name == "Neural Network":
    X_next_day_transformed = preprocessor.transform(X_next_day)
    next_day_pred = best_model.predict(X_next_day_transformed).flatten()
elif best_model_name == "SARIMA":
    # SARIMA forecast logic would go here
    print("SARIMA forecast requires special handling...")
    
else:
    # For scikit-learn models, we can use the pipeline directly
    next_day_pred = best_model.predict(X_next_day)

# Add predictions to the DataFrame
next_day_df['Predicted_Demand'] = next_day_pred

# Plot next day forecast
plt.figure(figsize=(14, 7))
plt.plot(next_day_df['DateTime'], next_day_df['Predicted_Demand'], 'b-', marker='o', label=f'Forecast ({best_model_name})')
plt.title(f'Next-Day Hourly Demand Forecast for {next_day.date()}')
plt.xlabel('Hour')
plt.ylabel('Demand (MW)')
plt.xticks(next_day_df['DateTime'], [f"{h}:00" for h in range(24)], rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('next_day_forecast.png')

# 12. Optional: Time Series Models (SARIMA)

try:
    print("\nTraining SARIMA model (this may take some time)...")

    # Prepare time series data for SARIMA
    
    ts_data = df_features[['DateTime', 'Demand (MW)']].set_index('DateTime')
    ts_data = ts_data.resample('H').mean()  # Ensure hourly frequency

    # Split into train/test
    ts_train = ts_data[ts_data.index <= split_date]
    ts_test = ts_data[ts_data.index > split_date]

    # Fit SARIMA model 
    
    sarima_model = SARIMAX(ts_train,
                           order=(1, 1, 1),
                           seasonal_order=(1, 1, 1, 24),  # 24-hour seasonality
                           enforce_stationarity=False,
                           enforce_invertibility=False)

    sarima_results = sarima_model.fit(disp=False)
    print("SARIMA model summary:")
    print(sarima_results.summary())

    # Make predictions
    sarima_pred = sarima_results.forecast(steps=len(ts_test))

    # Evaluate
    sarima_mae = mean_absolute_error(ts_test, sarima_pred)
    sarima_rmse = np.sqrt(mean_squared_error(ts_test, sarima_pred))
    sarima_mape = np.mean(np.abs((ts_test.values - sarima_pred) / ts_test.values)) * 100

    print(f"\nSARIMA Model Performance:")
    print(f"MAE: {sarima_mae:.2f} MW")
    print(f"RMSE: {sarima_rmse:.2f} MW")
    print(f"MAPE: {sarima_mape:.2f}%")

    # Calculate improvement over baseline
    sarima_improvement = ((naive_mae - sarima_mae) / naive_mae) * 100
    print(f"Improvement over naive baseline: {sarima_improvement:.2f}%")

    # Plot predictions
    plt.figure(figsize=(14, 7))
    plt.plot(ts_test.index, ts_test.values, label='Actual Demand', alpha=0.7)
    plt.plot(ts_test.index, sarima_pred, label='SARIMA Prediction', alpha=0.7)
    plt.title('SARIMA: Actual vs Predicted Demand')
    plt.xlabel('Date')
    plt.ylabel('Demand (MW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('sarima_prediction.png')

    # Add to results
    sarima_results_dict = {
        'model_name': 'SARIMA',
        'mae': sarima_mae,
        'rmse': sarima_rmse,
        'mape': sarima_mape,
        'r2': 0,  # R² not commonly used for time series
        'y_pred': sarima_pred,
        'improvement_over_naive': sarima_improvement
    }
    results.append(sarima_results_dict)
    predictions['SARIMA'] = sarima_pred

except Exception as e:
    print(f"Error training SARIMA model: {e}")
    pass

# 13. Conclusions 

print("\n==== Electricity Demand Forecasting Project Summary ====")
print(f"\nTotal observations: {len(df)}")
print(f"Time period: {df['DateTime'].min()} to {df['DateTime'].max()}")
print(f"\nBest performing model: {best_model}")

# Update results 
results_df = pd.DataFrame(results)

# Sort by MAE for final comparison
final_comparison = results_df.sort_values('mae')[['model_name', 'mae', 'rmse', 'mape', 'r2']]
print("\nFinal Model Comparison (sorted by MAE):")
print(final_comparison)





print("\n==== End of Project Summary ====")

print("\nAll analysis complete!")

In [ ]:
import joblib

# Save Linear Regression
joblib.dump(lr_model, 'linear_regression_model.pkl')

# Save Ridge Regression
joblib.dump(ridge_model, 'ridge_model.pkl')

# Save Random Forest
joblib.dump(rf_model, 'random_forest_model.pkl')

# Save Gradient Boosting
joblib.dump(gb_model, 'gradient_boosting_model.pkl')

# Save XGBoost
joblib.dump(xgb_model, 'xgboost_model.pkl')

# Save Stacking Regressor
joblib.dump(stacking_model, 'stacking_model.pkl')

# Save LSTM Model
lstm_model.save('lstm_model.h5')